In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import glob

%matplotlib inline
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothic'

# CSV 파일들을 저장할 빈 DataFrame을 생성


# data 폴더에 있는 모든 CSV 파일을 가져옵니다.
csv_files = glob.glob('./high50data/*.csv')
print(csv_files)

['./high50data\\000270.csv', './high50data\\000660.csv', './high50data\\000810.csv', './high50data\\003550.csv', './high50data\\003670.csv', './high50data\\005380.csv', './high50data\\005490.csv', './high50data\\005930.csv', './high50data\\005935.csv', './high50data\\006400.csv', './high50data\\009150.csv', './high50data\\010130.csv', './high50data\\012330.csv', './high50data\\015760.csv', './high50data\\017670.csv', './high50data\\018260.csv', './high50data\\024110.csv', './high50data\\028260.csv', './high50data\\032830.csv', './high50data\\033780.csv', './high50data\\034020.csv', './high50data\\034730.csv', './high50data\\035420.csv', './high50data\\035720.csv', './high50data\\047050.csv', './high50data\\051910.csv', './high50data\\055550.csv', './high50data\\066570.csv', './high50data\\068270.csv', './high50data\\086790.csv', './high50data\\096770.csv', './high50data\\105560.csv', './high50data\\138040.csv', './high50data\\207940.csv', './high50data\\316140.csv', './high50data\\3234

In [8]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from itertools import combinations
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import Huber
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import AdamW

stock_num_li = ['005930', '000660', '207940', '005935', '005380', '005490', '000270', '051910', '035420', '006400', '068270', '105560', '012330', '003670', '028260', '055550', '066570', '035720', '032830', '096770', '003550', '086790', '000810','033780', '017670', '015760', '138040', '034730', '009150', '018260', '329180', '010130', '047050', '323410', '316140', '024110', '034020', '352820']
# stock_num_li =['329180', '010130', '047050', '323410', '316140', '024110', '034020', '352820']
normal = 'Open High Low Close Volume '
M1 = 'ma10'
DU = 'high_low_diff bb_upper'
M1SP = 'ma10 S&P500'
M2VM = 'ma20 volumn_ma5'
SPU = 'S&P500 bb_upper'
M1K = 'ma10 %K'
M2SP = 'ma20 S&P500'
KOU = 'Kospi bb_upper'
KSPU = 'Kospi S&P500 bb_upper'

WINDOW_SIZE = 50
BATCH_SIZE = WINDOW_SIZE * 2
epoch_size = 30
patience = 5
learning_rate = 0.01

# 뽑아낸 최적화 특성열 7가지
colset_list = [KSPU, M1, DU, M1SP, SPU, M1K, M2SP, KOU]
result_pridict = []
scaler = MinMaxScaler()
for stock_num in stock_num_li:
  stock = pd.DataFrame()
  file_path  = [csv for csv in csv_files if f'./high50data\\{stock_num}.csv' in csv][0]
  df = pd.read_csv(file_path)
  stock = pd.concat([stock, df], ignore_index=True)

  for colset in colset_list:
    tmp_use_col = normal+colset
    colset = colset.replace(" ", "_")
    use_cols = tmp_use_col.split()
    ln_use_cols =len(use_cols)
    # print(ln_use_cols, use_cols)
    scaled = scaler.fit_transform(stock[use_cols])
    df = pd.DataFrame(scaled, columns=use_cols)


    X, y = [], []

    for i in range(WINDOW_SIZE, len(scaled)):
        X.append(scaled[i - WINDOW_SIZE:i])
        y.append([scaled[i][3]]) # Close 열만 학습
    X, y = np.array(X), np.array(y)

    split_ratio = 0.8
    split_index = int(split_ratio * len(X))

    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
# # '373220'
    # model = Sequential([
    #     GRU(32, activation='tanh', input_shape=(WINDOW_SIZE, ln_use_cols)),
    #     Dense(16, activation='relu'),
    #     Dense(ln_use_cols)
    # ])

    model = Sequential([
        GRU(32, activation='tanh', input_shape=(WINDOW_SIZE, ln_use_cols)),
        Dropout(0.2),  # Dropout 레이어 추가 (0.2는 드롭아웃 비율을 나타냅니다)
        Dense(16, activation='relu'),
        Dropout(0.2),  # Dropout 레이어 추가
        Dense(ln_use_cols)
    ])

    loss = Huber()
    # optimizer = Adam(learning_rate=learning_rate)
    # AdamW 옵티마이저를 생성
    optimizer = AdamW(
        learning_rate=learning_rate, #  가중치 및 편향(parameter)을 조정하는 데 사용되는 스칼라 값
        weight_decay=0.004,  # 가중치 감쇠
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07, # 0으로 나누지 않아지도록 매우 작은 값 설정
        amsgrad=False,
        clipnorm=None,  # 가중치의 기울기는 해당 노름이 이 값보다 높지 않도록 개별적으로 잘린다
        clipvalue=None, # 부동. 설정된 경우 각 가중치의 기울기가 이 값보다 높지 않도록 잘린다
        global_clipnorm=None,
        use_ema=False, # EMA는 모델 가중치의 지수 이동 평균(각 훈련 배치 후 가중치 값이 변경됨)을 계산하고 주기적으로 이동 평균으로 가중치를 덮어쓰는 것으로 구성
        ema_momentum=0.99, # 모델 가중치의 EMA를 계산할 때 사용할 운동량
        name="AdamW",
    )

    model.compile(loss=loss, optimizer=optimizer, metrics=[MeanAbsoluteError()])

    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    tmp = '_'.join(use_cols[5:])
    checkpoint = ModelCheckpoint(
        filepath=f'./checkpoint/GRU/AdamW/{WINDOW_SIZE}/{stock_num}_{tmp}_{WINDOW_SIZE}.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1,
    )

    history = model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        epochs=epoch_size,
        callbacks=[early_stopping, checkpoint],
        batch_size=BATCH_SIZE
    )

    input_scaled = stock[use_cols]
    total_data = [[input_scaled[len(input_scaled)-WINDOW_SIZE-1:len(input_scaled)-1]]]
    conform_total_data = [[input_scaled[len(input_scaled)-1:len(input_scaled)]]][:4]

    data = np.array(total_data)
    data = data.tolist()
    data = data[0]

    conform_data = conform_total_data[0][0]
    conform_data = conform_data.values[0]

    data_2d = np.array(data).reshape(-1, len(data[0][0]))

    scaler = MinMaxScaler()
    scaler.fit(data_2d)

    scaled_data_2d = scaler.transform(data_2d)

    scaled_data_3d = scaled_data_2d.reshape(len(data), len(data[0]), len(data[0][0]))

    model = load_model(f'./checkpoint/GRU/AdamW/{WINDOW_SIZE}/{stock_num}_{tmp}_{WINDOW_SIZE}.h5', custom_objects={'MeanAbsoluteError': MeanAbsoluteError()})

    predicted_value = model.predict(scaled_data_3d)
    # print(predicted_value)
        # 기존 모델에서 출력 레이어를 새로운 모델로 추출
    output_layer = model.get_layer(index=-1)

    # 새로운 모델을 생성하고 출력 레이어를 수정
    new_model = Model(inputs=model.input, outputs=output_layer.output)

    # 예측을 수행
    predicted_value = new_model.predict(scaled_data_3d)

    # 예측된 값 출력
    print(predicted_value)
    predicted_value_original = scaler.inverse_transform(predicted_value)
    predicted_value_original = np.round(predicted_value_original, ln_use_cols)[0][:4]
    conform_total_data = conform_total_data[0][0].values[0]
    predicted_value_original = list(predicted_value_original)
    predicted_value_original.extend(conform_total_data[:4])
    
    use_cols_check = '/'.join(use_cols)
    predicted_value_original.append(use_cols_check)
    result_use_cols = use_cols[:4]
    result_use_cols.extend(['_Open', '_High', '_Low', '_Close', 'Use_Cols_Check'])

    result_dict = dict(zip(result_use_cols, predicted_value_original))
    result_pridict.append(result_dict)



Epoch 1/10
32/34 [===========================>..] - ETA: 0s - loss: 0.0030 - mean_absolute_error: 0.0517
Epoch 1: val_loss improved from inf to 0.00122, saving model to ./checkpoint/GRU/AdamW/50\005930_['Kospi', 'S&P500', 'bb_upper']_50.h5
34/34 [==============================] - 5s 47ms/step - loss: 0.0029 - mean_absolute_error: 0.0505 - val_loss: 0.0012 - val_mean_absolute_error: 0.0387
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 7.7614e-04 - mean_absolute_error: 0.0266
Epoch 2: val_loss improved from 0.00122 to 0.00053, saving model to ./checkpoint/GRU/AdamW/50\005930_['Kospi', 'S&P500', 'bb_upper']_50.h5
34/34 [==============================] - 1s 31ms/step - loss: 7.7614e-04 - mean_absolute_error: 0.0266 - val_loss: 5.3048e-04 - val_mean_absolute_error: 0.0255
Epoch 3/10
32/34 [===========================>..] - ETA: 0s - loss: 5.2671e-04 - mean_absolute_error: 0.0212
Epoch 3: val_loss improved from 0.00053 to 0.00045, saving model to ./checkpoint/GRU/AdamW/

In [9]:
print(result_pridict)
import pandas as pd


# 데이터를 DataFrame으로 변환
df = pd.DataFrame(result_pridict)

# DataFrame을 엑셀 파일로 저장
df.to_excel('GRU_50.xlsx', index=False)

[{'Open': 78525.836, 'High': 78559.37, 'Low': 77458.79, 'Close': 78267.02, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/Kospi/S&P500/bb_upper'}, {'Open': 78487.72, 'High': 78249.86, 'Low': 77425.07, 'Close': 78411.086, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/ma10'}, {'Open': 77086.69, 'High': 77492.47, 'Low': 76755.766, 'Close': 77250.734, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/high_low_diff/bb_upper'}, {'Open': 78801.57, 'High': 78081.76, 'Low': 77212.695, 'Close': 78153.97, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/ma10/S&P500'}, {'Open': 77610.055, 'High': 78316.19, 'Low': 77668.055, 'Close': 77942.74, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols